In [6]:
import os 
import numpy as np
from scipy import integrate as scipy_integrate, LowLevelCallable
# Command to execute
cmd = 'cc -fPIC -shared -o lw_vimanic.so lw_rotate.c lw_vimanic.c  lw_tensor.c lw_vimanic_integrands.c'

# Using os.system() method
os.system(cmd)

0

In [7]:
# Access your C code
import ctypes
so_file = "./lw_vimanic.so"
c_functions = ctypes.CDLL(so_file)

In [8]:
def set_c():
    #c_functions.cset_c.restype =  ctypes.c_void
    c_functions.cset_c.argtypes = (ctypes.c_longdouble,)
    return c_functions.cset_c

In [9]:
def set_vc():
    #c_functions.cset_vc.restype =  ctypes.c_void
    c_functions.cset_vc.argtypes = (ctypes.c_longdouble,)
    return c_functions.cset_vc

In [10]:
def set_sphere_R():
    #c_functions.cset_sphere_R.restype =  ctypes.c_void
    c_functions.cset_sphere_R.argtypes = (ctypes.c_longdouble,)
    return c_functions.cset_sphere_R

In [11]:
def set_S():
    #c_functions.cset_S.restype =  ctypes.c_void
    c_functions.cset_S.argtypes = (ctypes.c_longdouble,)
    return c_functions.cset_S

In [12]:
def get_c():
    c_functions.cget_c.restype =  ctypes.c_longdouble
    c_functions.cget_c.argtypes = ()
    return c_functions.cget_c

In [13]:
def get_omega():
    c_functions.cget_omega.restype =  ctypes.c_longdouble
    c_functions.cget_omega.argtypes = ()
    return c_functions.cget_omega

In [14]:
def get_vc():
    c_functions.cget_vc.restype =  ctypes.c_longdouble
    c_functions.cget_vc.argtypes = ()
    return c_functions.cget_vc

In [15]:
def get_R_r():
    c_functions.cget_R_r.restype =  ctypes.c_longdouble
    c_functions.cget_R_r.argtypes = ()
    return c_functions.cget_R_r

In [16]:
def get_R_l():
    c_functions.cget_R_l.restype =  ctypes.c_longdouble
    c_functions.cget_R_l.argtypes = ()
    return c_functions.cget_R_l

In [17]:
def get_S():
    c_functions.cget_S.restype =  ctypes.c_longdouble
    c_functions.cget_S.argtypes = ()
    return c_functions.cget_S

In [18]:
def get_sphere_R():
    c_functions.cget_sphere_R.restype =  ctypes.c_longdouble
    c_functions.cget_sphere_R.argtypes = ()
    return c_functions.cget_sphere_R

In [19]:
set_c()(1.0)

919937488

In [20]:
get_c()()

1.0

In [21]:
get_vc()()

0.8

In [22]:
set_vc()(0.8)

0

In [23]:
get_omega()()

0.8

In [24]:
# радиус сферы интегрирования
R = 4 * get_R_l()() + 2 * get_S()()
R *= 1.5
# R = 40 * get_R_l() + 20 * get_S()

R

6.15000000000000

In [25]:
set_sphere_R()(R)
get_sphere_R()()

6.1499999999999995

In [26]:
T = np.float128(2*pi)/get_omega()()

In [27]:
def sum_Fy_Integrand():
    c_functions.sum_Fy_Integrand.restype =  ctypes.c_double
    c_functions.sum_Fy_Integrand.argtypes = (ctypes.c_double, ctypes.c_void_p)
    return c_functions.sum_Fy_Integrand

In [28]:
def Maxwells_stress_tensor_Integrand():
    c_functions.Maxwells_stress_tensor_Integrand.restype =  ctypes.c_double
    c_functions.Maxwells_stress_tensor_Integrand.argtypes = (ctypes.c_int, ctypes.POINTER(ctypes.c_double), ctypes.c_void_p)
    return c_functions.Maxwells_stress_tensor_Integrand

In [29]:
# интеграл компоненты силы по оси y, приложенной к зарядам, по периоду
int_fy = scipy_integrate.quad(LowLevelCallable(sum_Fy_Integrand()), 0, T)
int_fy

(-1.455904203057484, 8.170211071819577e-12)

In [30]:
%timeit scipy_integrate.quad(LowLevelCallable(sum_Fy_Integrand()), 0, T)

11 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
# интеграл компоненты силы по оси y, приложенной к зарядам, по периоду отнесённый к периоду
int_fy[0] / T

-0.18537148046789209862

In [32]:
nquad_opts_2 = { 'epsabs' : 1.0e-07,
                 'epsrel' : 1.0e-07,
                 'limit'  : 100 }

In [33]:
nquad_opts_2 = { 'epsabs' : 1.0e-02,
                 'epsrel' : 1.0e-04,
                 'limit'  : 100 }

In [ ]:
%timeit Maxwells_stress_tensor_R_upper_half_sphere = scipy_integrate.nquad( \
    LowLevelCallable(Maxwells_stress_tensor_Integrand()), \
        [ [0, np.float128(pi)/2],  [0, np.float128(2*pi)],  [0, T]], opts=nquad_opts_2)

In [ ]:
Maxwells_stress_tensor_R_upper_half_sphere = scipy_integrate.nquad( \
    LowLevelCallable(Maxwells_stress_tensor_Integrand()), \
        [ [0, np.float128(pi)/2],  [0, np.float128(2*pi)],  [0, T]], opts=nquad_opts_2)

In [ ]:
Maxwells_stress_tensor_R_upper_half_sphere

In [ ]:
Maxwells_stress_tensor_R_upper_half_sphere

In [ ]:
Maxwells_stress_tensor_R_upper_half_sphere

In [ ]:
Maxwells_stress_tensor_R_down_half_sphere = scipy_integrate.nquad( \
    LowLevelCallable(Maxwells_stress_tensor_Integrand()), \
        [ [np.float128(pi)/2, np.float128(pi)],  [0, np.float128(2*pi)],  [0, T]], opts=nquad_opts_2)

In [ ]:
Maxwells_stress_tensor_R_down_half_sphere

In [ ]:
Maxwells_stress_tensor_R_upper_half_sphere[0] + Maxwells_stress_tensor_R_down_half_sphere[0]

In [ ]:
Maxwells_stress_tensor_R = scipy_integrate.nquad(LowLevelCallable(Maxwells_stress_tensor_Integrand()) \
        [ [0, np.float128(pi)],  [0, np.float128(2*pi)],  [0, T]], opts=nquad_opts_2)


In [ ]:
Maxwells_stress_tensor_R